In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchfly.modules.transformers import CachedBertDecoderLM, ChineseBERTBaseConfig
from torchfly.text.tokenizers import BertTokenizer
from torchfly.utils import get_pretrained_states

In [2]:
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

In [3]:
model_states = get_pretrained_states("chinese-gpt-bert-small")

File exists: /home/wuqy1203/.cache/torchfly/models/chinese-gpt-bert-small.pth


In [4]:
model = CachedBertDecoderLM(ChineseBERTBaseConfig)
model.load_state_dict(model_states, strict=False)

<All keys matched successfully>

In [5]:
device = torch.device("cuda")
model = model.to(device)

In [6]:
def top_k_logits(logits, k):
    """Mask logits so that only top-k logits remain
    """
    values, _ = torch.topk(logits, k)
    min_values = values[:, -1].unsqueeze(1).repeat(1, logits.shape[-1])
    return torch.where(logits < min_values, torch.ones_like(logits, dtype=logits.dtype) * -1e10, logits)

In [7]:
prompt = tokenizer.encode("阿里巴巴集团宣布收购雅虎")
batch_size = 1

In [8]:
top_k = 50
temperature = 0.8
length = 0

start_predictions = torch.LongTensor([[101] + prompt]* batch_size).to(device)
mask = torch.ones(batch_size, start_predictions.shape[1]).to(device)

past = None

with torch.no_grad():
    # cache saves in past
    logits, past = model(start_predictions, mask, past=None, past_length=0)
    logits = logits[:, -1, :] / temperature
    logits = top_k_logits(logits, k=top_k)

    sentence = []

    probs = F.softmax(logits, dim=-1)
    prob, prev_pred = torch.topk(probs, k=1, dim=-1)
    sentence.append(prev_pred)
    length += 1

    # decoding loop
    for i in range(500):
        mask = F.pad(mask, (0, 1), "constant", 1.0)
        logits, past = model(prev_pred, mask, past=past, past_length=length)
        logits = logits.squeeze(1) / temperature
        logits = top_k_logits(logits, k=top_k)
        probs = F.softmax(logits, dim=-1)
        prev_pred = torch.multinomial(probs, num_samples=1)
        sentence.append(prev_pred)
        length += 1

    sentence = torch.cat(sentence, dim=-1)



In [9]:
tokenizer.decode(sentence[0].tolist())

'的目前在美国公司的重并购案，既有互联网行业公司的层次，也有在内容经营领域的全新尝试，也是一个大胆的尝试。如果两者都不能成功，一家公司可能将一个网站分裂出来，这将是一个比较大的尝试，因为自己的业务没有出现在《财经》周刊的头条新闻里。事实也表明，阿里巴巴有可能在这条新闻中获得一个非常可靠的人力资源专家团。对于雅虎的这种"人力资源不足"的情况，可以采取以下三种方法：第一、雅虎没有建立全面的内部网：《雅虎搜索已经成为中国内部网站的标签之一》，其实，这样做可能会有效地缓解网民的恐慌：每天的访问量越多，出现这种情况的几率就越大。所以，可以在前面的20万次的搜索结果中，加入一些自然词汇。第二、在《阿里的故事》这篇文章中，会让你意外的是，雅虎的用户通过这个文章，你可以查到上百条每天你会收到数千条自然词汇的内容，让你便秘的几率增加一倍。你大便很通畅可以大大减少，如果大便不成功，由于里面含有毒素，新浪知道的人体不能合成上述的有害物质。第三、通过《财经》周刊刊登的每天你可以看到很多关键词，内容包括，看新闻、看产品或食品，看自然在收。也可以用《商业计划书》、《政治经济》、《互联网管理》、《信息安全》。你可以在这些'